In [1]:
import numpy as np
import sys
import math
import itertools

# Programación dinámica

## 3.1 Introducción

La Programación Dinámica (PD), al igual que divide y vencerás (D&C, por sus siglas en inglés), resuelve problemas recursivamente, dividiéndolos en subproblemas y combinando sus soluciones.

Ahora bien, la naturaleza de los subproblemas que estos 2 métodos resuelven, son fundamentalmente distintas y, por lo tanto, la forma de resolverlos, también.
Como ya se ha dicho, los subproblemas de D&C son **disjuntos**. Por ejemplo, en _merge sort_ se divide un problema correspondiente a una lista de tamaño `n`, en 2 subproblemas de tamaño `n/2` que pueden ser resueltos independientemente y que serán utilizados para formar la solución al problema original.

Esta aproximación no daría la respuesta correcta a un problema que se resuelve por PD, como el de hallar el camino más rápido entre 2 puntos. Un clásico problema de optimización que usaremos de ejemplo y en el que nos abstraeremos de valores numéricos.

<img src="grilla.png" alt="mm" style="width: 1600px;"/>

Si se dividiera el problema original (**A4-D1**) en, por ejemplo, 2 subproblemas como pueden ser: **A4-C2** y **C2-D1**, la combinación de las soluciones óptimas a estos subproblemas, no será necesariamente la solución óptima al problema original, ya que existe la posibilidad que para ir de A4 a D1, no tenga q pasar por la celda C2. 

Será entonces que subidividimos mal al problema? Vemos que la solución pasa por C1, entonces quizás había que dividir **A4-D1** en **A4-C1** y **C1-D1**, pero, cómo podríamos haber sabido que el camino óptimo correspondía a la conjunción de esos 2 caminos? Tampoco podíamos descartar que pase por la celda **D2**, y por lo tanto subidividir el problema **A4-D1** en **A4-D2** y **D2-D1**.

En realidad, las 3 subidivisiones son necesarias. Para hallar el camino más rápido (camino óptimo), **A4-D1** es necesario hallar los caminos óptimos **A4-C1**, **A4-C2** y **A4-D2** (subproblemas **C1**, **C2**, **D2**), y luego ver cual de esos caminos es el más rápido, el de menor costo, es decir, el óptimo.
Luego, al buscar el camino óptimo entre estas 3 posibilidades haremos lo mismo, recursivamente, hasta el caso base, al igual que en D&C. Verdad?

<img src="1_grilla.png" alt="mm" style="width: 1600px;"/>

Ahora vemos que no sólo tenemos que obtener 3 subproblemas para luego resolver el problema original, sino que estos subproblemas no son independientes. Para resolver **C1** y **D2**, debemos resolver **C2**. Y también hay 1 subproblema de **C2** q será necesario p/ resolver **C1** y otro para **D2**. Los subproblemas están superpuestos. 

Si resolvieramos los subproblemas así como se nos presentan, estaríamos repitiendo cálculo. De nuevo, esto no ocurre en un algoritmo de D&C como _merge sort_, donde el ordenamiento de una porción de una lista, no contiene información alguna, ni depende, del ordenamiento de otra porción de la lista.

<img src="1_tree.png" alt="mm" style="width: 1200px;"/>

Este árbol continua hasta llegar al caso base **A4**, que representa el camino óptimo para llegar a **A4** desde **A4**, lo cual es una trivialidad, pero ésto no nos sorprende ya que así son los casos base.

#### Subproblemas de borde

Cabe también aclarar que no todo subproblema depende de la resolución de 3 subproblemas. Para llegar a las celdas de los bordes (columna **A**, fila **4**), solo hay un camino posible. Estos subproblemas no representan casos base en nuestra función recursiva, ya que su solución depende de un subproblema anterior (y que también es un subproblema de borde), pero por ser esta dependencia directa y única, la solución a estos subproblemas de borde es directa. Esta aclaración es importante ya que es común encontrar subproblemas de borde en algoritmos de PD.

-------------

## 3.2 Características de los problemas que se pueden resolver por PD

PD aplica en problemas de cálculo, decisión y optimización. En problemas de cálculo obtiene un valor, en problemas de decisión obtiene la decisión correcta y una solución correcta, y en problemas de optimización obtiene el costo óptimo (o _score_ óptimo) y una solución óptima. Por ser esta última aplicación la más común de ahora en más hablaremos de costo (_score_) óptimo y solución óptima.

### Subestructura óptima

Un problema tiene subestructura óptima si la solución óptima a sus subproblemas puede ser utilizada para obtener la solución óptima al problema original.
En el ejemplo anterior, cada subproblema dependía de 3 subproblemas anteriores (o posteriores, depende de como se lo vea), y la solución al subproblema original es la solución a uno de los subproblemas, más el paso adicional de ir desde una celda anterior a la que corresponde al subproblema actual.

El ordenamiento de una lista no tiene subestructura óptima. En _merge sort_, luego de ordenar las 2 mitades de una lista, el ordenamiento final no proviene de elegir alguna de las mitadas o de simplemente concatenar las 2 medias listas ordenadas. La solución final requiere una etapa de combinación, una intercalación de elementos. Esa necesidad de una combinación (etapa de _combine_), es lo que previene a los algoritmos que se resuelven por D&C, ser resueltos por PD.

Las diferencias entre problemas que tienen subestructura óptima son sutiles, y nos llevará tiempo reconocerlas. No sólo es necesario que un problema se pueda dividir en subproblemas, sino también es necesario que la solución de alguno de estos subproblemas sea parte de la solución del problema original, o visto al revés, que la solución del problema original contenga la solución a uno de sus subproblemas

### Superposición de subproblemas

Esta es la segunda característica que estos problemas presentan y ya vimos un ejemplo de esta superposición. 

Es esta característica la que permite el ahorro de cómputo de un algoritmo de PD, frente a uno bruto (_naive_) y la forma de evitar el recálculo de un mismo subproblema es mediante la utilización de una **estructura de datos** que almacene los resultados de los subproblemas para su posterior utilización. 

Esta estructura de datos contendrá las respuestas a todos los subproblemas, desde el caso base, hasta el problema original. Y el orden en el que un algoritmo de PD se propone llenar esta estructura de datos permite clasificarlo en 2 tipos de algoritmos de PD:  _top-down_ con memonización y _bottom-up_.

Estos algoritmos avanzan por el árbol de subproblemas en sentidos opuestos, y si bien veremos esta clasificación dentro del marco de PD, ésta aplica a todos los algoritmos que puedan expresarse con una función recursiva [citation needed]

### _top-down_

Estos algoritmos empiezan proponiéndose la resolución del problema original y luego van determinando, recursivamente, que subproblemas tienen que resolverse para resolver el subproblema original hasta llegar a un caso base, a partir del cual desanudan el camino, resolviendo cada subproblema.

Volviendo al ejemplo inicial, este algoritmo se propone, en primer lugar, resolver el problema de hallar el camino óptimo **A4-D1**, luego lo divide en subproblemas hasta proponerse la resolución del caso base: hallar el camino óptimo **A4-A4**.

Este procedimiento es propio de cualquier algoritmo recursivo (recordar _merge-sort_), lo que diferencia a los algoritmos de PD es el uso de la memonización.

#### memonización

Es el almacenamiento de valores óptimos y soluciones óptimas (es decir, resultados), de los subproblemas para posterior reutilización en la resolución de otro subproblema.

En cierta forma, es la respuesta intuitiva a la superposición de problemas de PD. Si un subproblema puede ser utilizado para resolver posteriores subproblemas, entonces lo natural es resolverlo y luego almacenar la respuesta en un _cache_ para posterior reutilización, como figura abajo.

Así, se inicializará una **estructura de datos** vacía con 1 elemento para cada subproblema. A cada subproblema resuelto su respuesta será almacenada en el lugar que le corresponda. Asimismo, cada vez que el algoritmo se proponga resolver un problema, intentará primero encontrar su respuesta en esta estructura de datos y así ahorrarse el recómputo.
El primer elemento de la estructura de datos en llenarse corresponderá al caso base, y el último a la respuesta del problema original.

### _bottom-up_

Estos algoritmos empiezan proponiéndose la resolución del subproblema más reducido, el trivial: el caso base. Luego reutiliza el resultado de este para resolver el siguiente subproblema y luego ambos resultados para resolver el siguiente, y así continua hasta llegar al problema orignal y resolverlo, al igual que con los anteriores subproblemas, reutilizando las respuestas anteriores.

Volviendo al ejemplo inicial, este algoritmo se propone, en primer lugar, resolver el problema de hallar el camino óptimo **A4-A4**, luego avanza con los siguientes subproblemas hasta proponerse la resolución del problema original: hallar el camino óptimo **A4-D1**. 

Una vez que se la conoce, esta aproximación parece ser la más natural de las 2.

Al empezar con el caso base y luego continuar con los subproblemas más pequeños hasta llegar al subproblema más grande (el original), estos algoritmos se aseguran tener todas las respuestas necesarias para resolver cada subproblema que se les presenta. 
Es decir, el primer subproblema que se proponen es el caso base y éste no requiere ninguna respueta anterior por su trivialidad. 
El segundo subproblema que se les presente, cualquiera sea éste, solo requiere la respuesta al caso base. El tercero, utilizará la respuesta de alguno de estos 2 subproblemas anteriores, y así continua. 
Esto podemos verlo en el ejemplo inicial.

#### advertencia

Nótese que las aproximaciones _top-down_ y _bottom-up_ varían en el orden en el que se **proponen** resolver los subproblemas y no en el orden de resolución de subproblemas. En ese caso, ambos resuelven de la misma y única posible manera: desde el caso base, hasta el problema original.

Las diferencias están en el sentido en el que avanzan por el árbol de subproblemas y en la forma de explorarlo: los _top-down_ lo hacen en profundidad y los _bottom-up_ lo hacen a lo ancho. Veremos más sobre esto de "profundidad y ancho" en siguientes clases.

<img src="2_tree.png" alt="mm" style="width: 2000px;"/>

## 3.3 _subset sum_

Apliquemos todos estos abstractos en un ejemplo bien concreto.

Dada una lista de enteros `conjunto` y un entero `suma`, determinar si existe un subconjunto dentro de `conjunto` cuya sumatoria sea igual a `suma`.

Veamos un ejemplo simple:

In [4]:
conjunto = [3, 20, 1, 12, 5, 2]
suma = 8

Es fácil ver que el subconjunto `[3, 5]` suma `8` y por lo tanto es una solución del problema.

Notar que este es un problema de decisión, debemos determinar si algo es posible o no (V o F). Entonces, lo que en optimización llamamos solución óptima, en este caso sería una solución correcta y en vez de un valor óptimo, tendríamos un valor que debe ser correcto. Por ejemplo, `[1, 5]` también es una solución, pero es incorrecta, ya que su valor `6`, es incorrecto.

También notamos una característica que suele aparecer también en los problemas de optimización, la multiplicidad de soluciones correctas posibles. `[1, 5, 2]` es otra solución correcta, ya que suma el valor correcto, `8`. Nótese que puede haber varias soluciones correctas, pero el valor correcto es único. Ya vimos el fenómeno análogo en problemas de optimización, distintas soluciones óptimas, pero un único valor óptimo, ya que si existe un máximo(mínimo), necesariamente éste es único, sino no sería el máximo (mínimo).

Ya que tenemos un ejemplo, vamos a empezar por un paso que no es estrictamente necesario, pero suele ser útil: pensaremos un algoritmo bruto que resuelva este problema.

Y el algoritmo más simple para este problema, es el que realiza todas las sumas posibles entre los números del conjunto, hasta hallar la combinación de elementos (subconjunto), correcta:

In [5]:
def subset_sum_bruto(conjunto, suma):
    for i in range(0, len(conjunto)+1):
        for subconjunto in itertools.combinations(conjunto, i):
            if sum(subconjunto) == suma:
                print("Match. Subconjunto: ", subconjunto)
                return
    print("No hay match.")
    return

No es necesario entender esta función. Mejor aún, ni la lean, ya que estoy rompiendo una de las reglas de la clase (no utilizar librerías mágicas de Python).

Veamos si funciona:

In [6]:
subset_sum_bruto(conjunto, suma)

Match. Subconjunto:  (3, 5)


Funciona, y si bien no encuentra todas las respuestas, nos devuelve el valor correcto (match), y la solución correcta (Subconjunto). Probá variando el valor de `suma` para ver distintos resultados.

Vemos que hace exactamente este algoritmo.

Genera todos los subconjuntos posibles y los evalua uno por uno. Para nuestro ejemplo, ésta sería una descripción de los subconjuntos posibles:

## 3.3 Pasos en la solución de un algoritmo de PD

In [ ]:
Caracterizar la estructura de una solución óptima
determinar que variables caracterizan a un estado, es decir, diferencian un subproblema de otro. Éstas aparecerán como argumentos de nuestra función


Definir la solución óptima recursivamente
Aquí debe ser evidente la superposición de SP


Obtener el costo (o score) de la solución óptima


Construir la solución óptima
A veces esto puede hacerse intercalando código en la función original. Otras veces es más simple tomar la estructura de datos con los valores óptimos de los SPs resueltos y de ahí determinar la solución óptima.


### esquematico


Pienso variables que pueden caracterizar a un subproblema (estarán incluídas en los parámetros de la función q escribiré más adelante).


Dibujo arbolito de subproblemas con esas variables.


Pienso casos base.


Escribo mi recursión.


### pseudocódigo


Escribo mi algoritmo recursivo.


Top-down con memonización p/ hallar costo de solución óptima.


Bottom-up p/ hallar costo de solución óptima.


Hallar solución óptima, ya sea intercalando código en la solución original, o agregando una rutina que recupere la solución óptima.


("Programación" en este contexto se refiere
a un método tabular, no a escribir código de computadora.) Como vimos en los Capítulos 2
y 4, los algoritmos de divide y vencerás dividen el problema en inconexos.
lemas, resuelva los subproblemas de forma recursiva y luego combine sus soluciones para resolver
el problema original. Por el contrario, la programación dinámica se aplica cuando el subproblema
los lemas se superponen, es decir, cuando los subproblemas comparten subproblemas. En este contexto,
un algoritmo de divide y vencerás hace más trabajo del necesario, resolviendo repetidamente
ing los subproblemas comunes. Un algoritmo de programación dinámica resuelve cada
subproblema solo una vez y luego guarda su respuesta en una tabla, evitando así la
trabajo de recalcular la respuesta cada vez que resuelve cada subproblema.
Normalmente aplicamos programación dinámica a los problemas de optimización. Tal problema
Los lemas pueden tener muchas soluciones posibles. Cada solución tiene un valor y deseamos
encuentre una solución con el valor óptimo (mínimo o máximo). Llamamos a tal
solución una solución óptima al problema, en contraposición a la solución óptima,
ya que puede haber varias soluciones que logren el valor óptimo.
Al desarrollar un algoritmo de programación dinámica, seguimos una secuencia de
cuatro pasos:

1. Caracterizar la estructura de una solución óptima.
2. Defina de forma recursiva el valor de una solución óptima.
3. Calcule el valor de una solución óptima, normalmente de forma ascendente.
4. Construya una solución óptima a partir de información calculada.

Los pasos 1 a 3 forman la base de una solución de programación dinámica a un problema. Si nosotros
solo necesitamos el valor de una solución óptima, y no la solución en sí, entonces
puede omitir el paso 4. Cuando realizamos el paso 4, a veces mantenemos
información durante el paso 3 para que podamos construir fácilmente una solución óptima.

Aunque acabamos de trabajar con dos ejemplos de la programación dinámica
método, es posible que todavía se pregunte cuándo se aplica el método. De un en-
perspectiva de ingeniería, ¿cuándo deberíamos buscar una solución de programación dinámica?
a un problema? En esta sección, examinamos los dos ingredientes clave que debe tener un problema de optimización para que se aplique la programación dinámica: óptimo
subestructura y subproblemas superpuestos. También revisamos y discutimos más a fondo
cómo la memorización podría ayudarnos a aprovechar los subproblemas superpuestos
propiedad en un enfoque recursivo de arriba hacia abajo.

### _subset sum_

### Fórmula recursiva

Subestructura óptima


El primer paso para resolver un problema de optimización mediante programación dinámica es
caracterizar la estructura de una solución óptima. Recuerde que presenta un problema
subestructura óptima si una solución óptima al problema contiene dentro de ella
Soluciones incorrectas a subproblemas. Siempre que un problema presente una subestructura óptima,
tenemos una buena pista de que podría aplicarse la programación dinámica. (Como el Capítulo 16 dis-
maldiciones, también podría significar que se aplica una estrategia codiciosa).
programación, construimos una solución óptima al problema a partir de soluciones óptimas
a los subproblemas. En consecuencia, debemos tener cuidado de asegurar que la gama de sub-
Los problemas que consideramos incluyen los que se utilizan en una solución óptima.
Descubrimos la subestructura óptima en los dos problemas que hemos examinado
en este capítulo hasta ahora. En la Sección 15.1, observamos que la forma óptima de cortar
levantar una varilla de longitud n (si es que hacemos algún corte) implica cortar de manera óptima
hasta las dos piezas resultantes del primer corte. En la Sección 15.2, observamos que
un paréntesis óptimo de Ai Ai C1 A j que divide el producto entre Ak
y AkC1 contiene en su interior soluciones óptimas a los problemas de paréntesis
Ai Ai C1 Ak y AkC1 AkC2 A j.
Se encontrará siguiendo un patrón común en el descubrimiento de sub-
estructura:

1. Muestra que una solución al problema consiste en tomar una decisión, como
elegir un corte inicial en una varilla o elegir un índice en el que dividir la matriz
cadena. Hacer esta elección deja uno o más subproblemas por resolver.
2. Supone que para un problema dado, se le da la opción que conduce a una
solucion optima. Todavía no te preocupas por cómo determinar este
elección. Simplemente asume que se te ha dado.
3. Dada esta opción, usted determina qué subproblemas surgen y cuál es la mejor manera de
caracterizar el espacio resultante de subproblemas.
4. Demuestra que las soluciones a los subproblemas utilizadas dentro de una solución óptima
al problema deben ser óptimos mediante el uso de una tecnología de "cortar y pegar"
nique. Lo hace suponiendo que cada una de las soluciones del subproblema no es
óptimo y luego derivar una contradicción. En particular, al "cortar" el
solución no óptima para cada subproblema y "pegando" el óptimo, usted
demostrar que se puede obtener una mejor solución al problema original, por lo que se contradice
suponer que ya tenía una solución óptima. Si una solución óptima da lugar a más de un subproblema, normalmente son tan similares
que puede modificar el argumento cortar y pegar para que uno se aplique a los demás
con poco esfuerzo

Para caracterizar el espacio de los subproblemas, una buena regla empírica dice tratar de
Mantenga el espacio lo más simple posible y luego amplíelo según sea necesario. Por ejemplo,
el espacio de subproblemas que consideramos para el problema del corte de varilla contenía
los problemas de cortar de manera óptima una varilla de longitud i para cada tamaño i. Este sub-
El espacio problemático funcionó bien, y no tuvimos necesidad de probar un espacio más general de
subproblemas.
Por el contrario, suponga que hemos intentado restringir nuestro espacio de subproblemas para
multiplicación de cadenas de matrices a productos de matrices de la forma A1 A2 A j. Como antes,
un paréntesis óptimo debe dividir este producto entre Ak y AkC1 para algunos
1 k <j. A menos que podamos garantizar que k siempre es igual a j 1, encontraríamos
que teníamos subproblemas de la forma A1 A2 Ak y A kC1 AkC2 A j, y que
este último subproblema no es de la forma A1 A2 A j. Para este problema, necesitábamos
permitir que nuestros subproblemas varíen en "ambos extremos", es decir, permitir que tanto i como j
varían en el subproblema Ai Ai C1 A j.
La subestructura óptima varía entre los dominios del problema de dos maneras:

1. cuántos subproblemas utiliza una solución óptima al problema original, y
2. cuántas opciones tenemos para determinar qué subproblema (s) usar en un
solucion optima.

## Construyendo solución óptima

### _top-down_ con memonización

Como vimos para el problema de corte por vara, hay un enfoque alternativo para Dy-
La programación namica que a menudo ofrece la eficiencia de la dinámica de abajo.
Enfoque de programación al tiempo que contiene una estrategia de arriba hacia abajo. La idea es
Memoice el algoritmo natural, pero ineficiente, recursivo. Como en la parte de abajo hacia arriba.
Proach, mantenemos una mesa con soluciones de subproblem, pero la estructura de control.
Para llenar la tabla es más como el algoritmo recursivo.
Un algoritmo recursivo memorizado mantiene una entrada en una tabla para la solución para
Cada subproblema. Cada entrada de tabla se contiene inicialmente al valor especial para indicar que
La entrada aún no se ha llenado. Cuando el subproblema se encuentra primero como el
El algoritmo recursivo se desarrolla, su solución se calcula y luego se almacena en la tabla.
Cada tiempo posterior que encontramos este subproblema, simplemente buscamos el
Valor almacenado en la tabla y devolverlo.

## _bottom-up_

En la práctica general, si todos los subproblemas deben resolverse al menos una vez, una parte inferior
Algoritmo de programación dinámica generalmente supera a superar la parte superior correspondiente
algoritmo memorizado por un factor constante, porque el algoritmo de abajo hacia arriba no tiene
Por encima de la recursión y menos gastos generales para mantener la tabla. Además, por
Algunos problemas podemos explotar el patrón regular de accesos de mesa en la dinámica.
Algoritmo de programación para reducir los requisitos de tiempo o espacio aún más. Alterar-
de forma nativa, si algunos subproblemas en el espacio del subproblema no deben resolverse en absoluto,
La solución memoIcada tiene la ventaja de resolver solo aquellos subproblemas que
son definitivamente requeridos.

------------

## 3.2

In [ ]:
## Acá, búsqueda binaria!

In [2]:
def bb_1(lista, valor, offset):
    if(len(lista) > 0):
        medio = len(lista) // 2

        if (valor < lista[medio]):
            return bb_1(lista[0:medio], valor, offset)
        elif (valor > lista[medio]):
            return bb_1(lista[medio+1:], valor, medio + offset + 1)
        else:
            return medio + offset
    else:
        return -1

In [8]:
# Mis inputs
lista = list(range(0, 100, 2))
valor = 41

# Tupla de salidas
(bb_1(lista, valor, 0), bb_2(lista, valor, 0, len(lista)))

(-1, -1)

Ahora, si dividir la lista en 2 mitades nos permitió descartar el 50%, por qué no hacerlo en tercios? Eso nos permitiría descartar el 66% de la lista en cada intento! Cómo sería el pseudocódigo de esta función? Pausen el video y dense la oportunidad de resolverlo.

----------

## 3.3

Veamos la recurrencia de la búsqueda binaria

$$ T(n) = 
 \begin{cases}
\Theta(1), \qquad \qquad \quad  \text{if} \ n == 1
\newline
T(\frac{n}{2}) + \Theta(1), \qquad \ \text{if} \ n > 1
\end{cases} $$

Decimos que ésta fórmula es abierta, ya que la recurrencia depende de si misma. Como vimos, la sumatoria sigue y, en principio, no podemos acotarla.

------------------------------------------------------------------------------------------

## 3.4


$$ T(n) = aT(\frac{n}{b}) + f(n) $$


------------

## 3.5 

`búsqueda_desordenada_recursiva()`

------------------------------------------------------------------------------------------

<img src="merge_sort_3.png" alt="merge_sort_3" style="width: 600px;"/>

-------------------

## 3.8 Conclusión

* Pasos de PD: **recursiva**, **estructura de datos**
* **top-down**, **memonización**,
* **bottom-up**
* advertencia respecto a leer de cualquier página de internet.
* Recuerden leer el Cormen: **p(359-403)**
* Responder cuestionario de campus.

--------------------

### Contenidos a explicar durante la práctica

1. coeficiente binomial
2. conjunto de partes (_power set_)
3. problema entero de la mochila en PD
4. complejidad pseudopolinomial
5. impacto (constante) de las llamadas recursivas de los _top down_ en su tiempo de ejecución y ventajas de estos frente a los _bottom up_ cuando no es necesario calcular todos los subproblemas
6. optimizaciones espaciales sobre la estructura de datos

## Extra

Hasta 2017, Clang implementaba su std::sort con quicksort recursivo!:

https://www.youtube.com/watch?v=Lcz0ZHewkHs